<a href="https://colab.research.google.com/github/rrfsantos/Large-Language-Model-Agents-MOOC-Fall-2024/blob/main/AI%20Agentic%20Design%20Patterns%20with%20AutoGen/L2_Sequential_Chats_and_Customer_Onboarding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -r /content/drive/MyDrive/AutoGen/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.1/257.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [13]:
from dotenv import load_dotenv
import warnings
import os
from autogen import ConversableAgent, initiate_chats
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [5]:
# Load environment variables from .env file
load_dotenv("/content/drive/MyDrive/AutoGen/env")
llm_config = {"config_list": [{"model": "gpt-3.5-turbo", "api_key" : os.getenv("OPENAI_API_KEY")}]}

## Creating the needed agents

In [6]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE'
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 10-28 15:52:02] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [7]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 10-28 15:52:09] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [8]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 10-28 15:52:49] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [9]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [10]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message":
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message":
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [14]:
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Reanata Rio de janeiro
customer_proxy_agent (to Onboarding Personal Information Agent):

Reanata Rio de janeiro

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Great! Thank you for providing your name and location. Is there anything else I can assist you with today, or are you all set for now?

------------------------------

## Print out the summary

In [15]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Reanata",
  "location": "Rio de Janeiro"
}


The person is from Rio de Janeiro and is interested in reading about cars.


Renata from Rio de Janeiro may enjoy reading about the Rio de Janeiro Grand Prix, a popular car race held in the city. It is an exciting event that attracts top drivers and car enthusiasts from around the world.




## Print out the cost

In [16]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000178, 'gpt-3.5-turbo-0125': {'cost': 0.000178, 'prompt_tokens': 206, 'completion_tokens': 50, 'total_tokens': 256}}, 'usage_excluding_cached_inference': {'total_cost': 0.000178, 'gpt-3.5-turbo-0125': {'cost': 0.000178, 'prompt_tokens': 206, 'completion_tokens': 50, 'total_tokens': 256}}}


{'usage_including_cached_inference': {'total_cost': 5.8e-05, 'gpt-3.5-turbo-0125': {'cost': 5.8e-05, 'prompt_tokens': 71, 'completion_tokens': 15, 'total_tokens': 86}}, 'usage_excluding_cached_inference': {'total_cost': 5.8e-05, 'gpt-3.5-turbo-0125': {'cost': 5.8e-05, 'prompt_tokens': 71, 'completion_tokens': 15, 'total_tokens': 86}}}


{'usage_including_cached_inference': {'total_cost': 0.00042350000000000005, 'gpt-3.5-turbo-0125': {'cost': 0.00042350000000000005, 'prompt_tokens': 328, 'completion_tokens': 173, 'total_tokens': 501}}, 'usage_excluding_cached_inference': {'total_cost': 0.00042350000000000005, 'gpt-3.5-turbo-0125': {'cost': 0.0004